In [1]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 1 # Always reload before execution only modules imported with '%aimport'
# %aimport pipeline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import os
import sys
import itertools
sys.path.append('src/')
import nn
import process_data
import nibabel as nib
from math import floor, ceil

from sklearn.metrics import confusion_matrix
import scipy.sparse
from scipy.misc import imrotate, imresize
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import rotate
from skimage import exposure
from skimage.io import imread, imsave
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print(local_device_protos)

import pipeline
import Unet

/home/jessica/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11846373283778878909
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11970700903
locality {
  bus_id: 1
}
incarnation: 13866162097602048981
physical_device_desc: "device: 0, name: TITAN Xp, pci bus id: 0000:02:00.0"
]


In [2]:
def tensorboard():
    import webbrowser
    webbrowser.open('http://localhost:6006')

    !tensorboard --logdir="logs"

In [3]:
models_dir = '/media/jessica/Storage/models/u-net_v1-0'
model_name = 'new_train_test'

In [4]:
training_data_dir = "/home/jessica/Documents/hart-seg-ml/training_data"

In [5]:
raw_data_lst, seg_data_lst = pipeline.load_all_data(training_data_dir)

In [6]:
x_train, x_val, x_test, y_train, y_val, y_test = pipeline.split_data(raw_data_lst, seg_data_lst, 60, 10, 30)

In [7]:
mean = 0
weight_decay = 1e-6
learning_rate = 1e-4

In [24]:
import importlib
tf.summary.FileWriterCache.clear()
importlib.reload(Unet)
importlib.reload(nn)
tf.reset_default_graph()
sess = tf.Session()
model = Unet.Unet(mean, weight_decay, learning_rate, dropout=0.5)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=1, keep_checkpoint_every_n_hours=1)

In [26]:
nn.train(sess, model, saver, x_train, y_train, x_val, y_val, x_test, y_test, epochs = 100, batch_size = 1, models_dir=models_dir, model_name=model_name, summary_writer=1)

1
Epoch 0 | Iter 2 | Loss: 0.9213 | Data: 2/3 | Time 1.0      
1
Epoch 1 | Iter 2 | Loss: 0.6318 | Data: 2/3 | Time 0.97     
1
Epoch 2 | Iter 2 | Loss: 0.5601 | Data: 2/3 | Time 0.98     
1
Epoch 3 | Iter 2 | Loss: 0.5234 | Data: 2/3 | Time 0.97     
1
Epoch 4 | Iter 2 | Loss: 0.4409 | Data: 2/3 | Time 0.98     
1
Epoch 5 | Iter 2 | Loss: 0.3786 | Data: 2/3 | Time 0.97     
1
Epoch 6 | Iter 2 | Loss: 0.3496 | Data: 2/3 | Time 1.0      
1
Epoch 7 | Iter 2 | Loss: 0.1943 | Data: 2/3 | Time 0.98     
1
Epoch 8 | Iter 2 | Loss: 0.1778 | Data: 2/3 | Time 0.97     
1
Epoch 9 | Iter 2 | Loss: 0.1078 | Data: 2/3 | Time 0.99     
1
Epoch 10 | Iter 2 | Loss: 0.06945 | Data: 2/3 | Time 0.99     
1
Epoch 11 | Iter 2 | Loss: 0.05798 | Data: 2/3 | Time 0.97     
1
Epoch 12 | Iter 2 | Loss: 0.05346 | Data: 2/3 | Time 0.99     
1
Epoch 13 | Iter 2 | Loss: 0.03944 | Data: 2/3 | Time 0.98     
1
Epoch 14 | Iter 2 | Loss: 0.02649 | Data: 2/3 | Time 0.99     
1
Epoch 15 | Iter 2 | Loss: 0.0204 | Data: 2/

In [ ]:
print(nn.validate(sess, model, x_test, y_test))

In [ ]:
pipeline.save_model(models_dir, model_name, saver, sess)